# 1. Setup your virtual environment

## For Mac

```bash
python -m venv .venv
source .venv/bin/activate
```
## For Windows 

```bash
python -m venv .venv
.venv\Scripts\activate
```

# Install the dependencies
```bash
pip install -r requirements.txt
```

We are going to use the following core libraries: 

- FastAPI
- Lanchain
- Langchain-openai
- Pydantic
- Uvicorn 

In [ ]:
# Define Pydantic Models 
from pydantic import BaseModel, Field
from typing import List

class Activity(BaseModel):
  """
  The activity data 
  """
  name: str
  location: str 
  category: List[str]
  description: str 

class KeyNote(BaseModel):
  """
  Key note taken by the KeyNoteTaker agent 
  """
  title: str = Field(description="This should be a short title that captures the key points of the note")
  note: str = Field(description="This should be the content of the note")


class UserProfile(BaseModel):
  """
  The user profile 
  """
  name: str = Field(description="The name of the user")
  age: int = Field(description="The age of the user")
  gender: str = Field(description="The gender of the user")
  interests: List[str] = Field(description="The interests of the user")
  location: str = Field(description="The location of the user")
  budget: float = Field(description="The budget of the user") 

In [ ]:
class AgentState(BaseModel):
  

# Agentic LLM

In [ ]:
# Initialize the agents 
# Summarisation Agent
def create_summarisation_agent():
    def summarize(state: AgentState) -> AgentState:
        # Get latest discussion
        # latest_discussion = state.discussions
        llm_summary = AzureChatOpenAI(
            azure_endpoint=os.environ['AZURE_OPENAI_ENDPOINT'],
            api_key=os.environ['AZURE_OPENAI_API_KEY'],
            deployment_name=os.environ['AZURE_OPENAI_DEPLOYMENT_NAME'],
            model_name=os.environ['AZURE_OPENAI_MODEL_NAME'],
            api_version=os.environ['AZURE_OPENAI_API_VERSION'],
            temperature=0,
            max_tokens=1000,
            timeout=None,
            max_retries=2,
        )
        logging.info(f"LLM model initialized")

        messages = [
            SystemMessage(content="""You are a discussion summarizer. 
            Create a concise summary of the discussion, extracting key points 
            and insights from the answers. Focus on patterns, unique perspectives, 
            and potential areas for deeper exploration."""),
            
            HumanMessage(content=f"""
            Question discussed: {state.current_question}
            
            Answers from participants:
            {state.discussions}
            
            Provide a short summary of the discussion <max 50 words>.
            """)
        ]

        llm_structured_output_summary = llm_summary.with_structured_output(SummarisationResponse)
        response = llm_structured_output_summary.invoke(messages)
        logging.info(f"response create_summarisation_agent: {response}")
        print("response create_summarisation_agent: ", response)
        timestamp = str(datetime.now().isoformat())
        state.summaries.append({timestamp: response.summary})
        return state
    return summarize